In [1]:
# import random
import torch
from card_engine import Card_Game

/home/txmszlou/Work/erdos-workshop/deep-learning/dqn-trick-game/utils/card_engine.py:29: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  def __init__(self, num_players=torch.tensor(4), num_cards=torch.tensor(13), trump=torch.tensor(3)):


In [9]:
# a random agent that choose a random legal move and plays it
# returns the chosen card to play : 0-51
def random_agent(game):
    moves = game.get_legal_moves()
    i = torch.randint(len(moves), (1,))
    chosen_move = moves[i]
    # game.play_card(chosen_move)
    return chosen_move

# a card playing environment that maintains a game environment and is responsible for
# using some agent to get to the next state from the current state, and computes the reward
class Card_Env:
    def __init__(self, num_players=torch.tensor(4), num_cards=torch.tensor(13), trump=torch.tensor(3), foreign_policy=random_agent):
        self.game = Card_Game(num_players, num_cards, trump)
        # foreign_policy : game -> deck_index
        self.foreign_policy = foreign_policy
    
    # return observation, reward, terminated
    def step(self, deck_index):
        # step to the next state using the given foreign policy to play three turns in the game
        # TODO: if try to step through an illegal move, the game ends immediately
        #       may want to modify in the future

        current_player = self.game.current_player
        current_tricks_won = self.game.tricks_won[current_player]

        # first play the current move
        if not self.game.is_move_legal(deck_index):
            print('player plays an illegal move')
            return None, 0, True
        
        self.game.play_card(deck_index)

        # let the next three players play using the foreign_policy
        for i in range(3):
            move = self.foreign_policy(self.game)
            if not self.game.is_move_legal(move):
                return None, 0, True
            self.game.play_card(move)

        # reward is 1 if the player won this trick, 0 otherwise
        reward = 1 if self.game.tricks_won[current_player] > current_tricks_won else 0
        
        return self.game.get_network_input, reward, False

In [21]:
card_env = Card_Env()

for x in range(13):
    move = random_agent(card_env.game)
    print(f'player move: {move}')
    _, reward, is_terminated = card_env.step(move)

    print(f'player reward: {reward}')

    if is_terminated:
        print('game ends')
        break

card_env.game.tricks_won


player move: tensor([18])
player reward: 0
player move: tensor([5])
player reward: 0
player move: tensor([24])
player reward: 0
player move: tensor([43])
player reward: 0
player move: tensor([50])
player reward: 0
player move: tensor([36])
player reward: 0
player move: tensor([33])
player reward: 0
player move: tensor([10])
player reward: 0
player move: tensor([19])
player reward: 0
player move: tensor([44])
player reward: 0
player move: tensor([49])
player reward: 0
player move: tensor([21])
player reward: 0
player move: tensor([16])
player reward: 0


tensor([4, 3, 5, 1])

In [2]:
game = Card_Game()


for x in range(52):
    moves = game.get_legal_moves()
    i = torch.randint(len(moves), (1,))
#     print(game.turn_counter, i, moves[i])
    print(game.turn_counter, game.current_player)
    chosen_move = moves[i]
    game.play_card(chosen_move)

game.tricks_won

tensor(0) tensor(0)
tensor(1) tensor(1)
tensor(2) tensor(2)
tensor(3) tensor(3)
tensor(4) tensor(1)
tensor(5) tensor(2)
tensor(6) tensor(3)
tensor(7) tensor(0)
tensor(8) tensor(3)
tensor(9) tensor(0)
tensor(10) tensor(1)
tensor(11) tensor(2)
tensor(12) tensor(1)
tensor(13) tensor(2)
tensor(14) tensor(3)
tensor(15) tensor(0)
tensor(16) tensor(2)
tensor(17) tensor(3)
tensor(18) tensor(0)
tensor(19) tensor(1)
tensor(20) tensor(3)
tensor(21) tensor(0)
tensor(22) tensor(1)
tensor(23) tensor(2)
tensor(24) tensor(1)
tensor(25) tensor(2)
tensor(26) tensor(3)
tensor(27) tensor(0)
tensor(28) tensor(3)
tensor(29) tensor(0)
tensor(30) tensor(1)
tensor(31) tensor(2)
tensor(32) tensor(0)
tensor(33) tensor(1)
tensor(34) tensor(2)
tensor(35) tensor(3)
tensor(36) tensor(0)
tensor(37) tensor(1)
tensor(38) tensor(2)
tensor(39) tensor(3)
tensor(40) tensor(1)
tensor(41) tensor(2)
tensor(42) tensor(3)
tensor(43) tensor(0)
tensor(44) tensor(3)
tensor(45) tensor(0)
tensor(46) tensor(1)
tensor(47) tensor(2)
te

/home/txmszlou/Work/erdos-workshop/deep-learning/dqn-trick-game/utils/card_engine.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  card = torch.nn.functional.one_hot(torch.tensor(deck_index, dtype=int), self.num_cards * self.num_players).flatten()


tensor([2, 4, 1, 6])

In [4]:
len(game.get_network_input())

3016

In [3]:
game.state.nonzero()[:,1].unflatten(0, (13, 4))

tensor([[ 5,  3, 37, 11],
        [38, 33, 36, 27],
        [10,  9, 32, 12],
        [ 4, 18, 25,  0],
        [ 7, 20, 29,  6],
        [35, 26, 34, 31],
        [ 8, 47, 23, 19],
        [46, 44, 45, 50],
        [48, 41, 51,  1],
        [24, 21, 17, 15],
        [22, 13, 43, 39],
        [40, 49, 30, 14],
        [28, 16,  2, 42]])